'''
#Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.
**Summary of the algorithm**
- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
**Data download**
[English to French sentence pairs.
](http://www.manythings.org/anki/fra-eng.zip)
[Lots of neat sentence pairs datasets.
](http://www.manythings.org/anki/)
**References**
- [Sequence to Sequence Learning with Neural Networks
   ](https://arxiv.org/abs/1409.3215)
- [Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    ](https://arxiv.org/abs/1406.1078)
'''

In [ ]:
!pip install keras

In [6]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate
import numpy as np

In [7]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'


In [8]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
print(input_characters)
target_characters = sorted(list(target_characters))
print(target_characters)
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
# print([len(txt) for txt in input_texts])
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

[' ', '!', '$', '%', '&', "'", ',', '-', '.', '0', '1', '2', '3', '5', '6', '7', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['\t', '\n', ' ', '!', '$', '%', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '5', '8', '9', ':', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '\xa0', '«', '»', 'À', 'Ç', 'É', 'Ê', 'à', 'â', 'ç', 'è', 'é', 'ê', 'ë', 'î', 'ï', 'ô', 'ù', 'û', 'œ', '\u2009', '’', '\u202f']
Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [9]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
# print(input_token_index)
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

t = np.zeros((3,3,3))
# print(t)

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


print(np.argmax(decoder_input_data[0], axis=-1))
print(np.argmax(decoder_target_data[0], axis=-1))

[ 0 43 45  2  3  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0]
[43 45  2  3  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0]


In [45]:
from keras.layers import Add 

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = Bidirectional(LSTM(latent_dim, return_sequences=True, return_state=True))
encoder2 = Bidirectional(LSTM(latent_dim, return_state=True))

lstm_outputs, _, _, _, _ = encoder_lstm(encoder_inputs)
encoder_outputs, f_state_h, f_state_c, b_state_h, b_state_c = encoder2(lstm_outputs)
# We discard `encoder_outputs` and only keep the states.

# encoder_states = [state_h, state_c]
# encoder_states = [f_state_h, f_state_c, b_state_h, b_state_c]
# encoder_states = [f_state_h, f_state_c]

state_h = Add()([f_state_h, b_state_h])
state_c = Add()([f_state_c, b_state_c])
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
epochs=20
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

# Save model
model.save('s2s.bi.h5')




Train on 8000 samples, validate on 2000 samples
Epoch 1/20
8000/8000 [==============================] - 94s 12ms/step - loss: 0.9096 - val_loss: 0.9243
Epoch 2/20
8000/8000 [==============================] - 68s 9ms/step - loss: 0.7075 - val_loss: 0.7295
Epoch 3/20
 704/8000 [=>............................] - ETA: 59s - loss: 0.6132

KeyboardInterrupt: 

In [38]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [39]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [40]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    print(input_seq.shape)
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence  :', input_texts[seq_index])
    print('Target sentence :', target_texts[seq_index][1:-1])
    print('Decoded sentence:', decoded_sentence)
    

(1, 16, 70)
-
Input sentence  : Go.
Target sentence : Va !
Decoded sentence: Alleur !

(1, 16, 70)
-
Input sentence  : Hi.
Target sentence : Salut !
Decoded sentence: Axtends !

(1, 16, 70)
-
Input sentence  : Hi.
Target sentence : Salut.
Decoded sentence: Axtends !

(1, 16, 70)
-
Input sentence  : Run!
Target sentence : Cours !
Decoded sentence: À lu !

(1, 16, 70)
-
Input sentence  : Run!
Target sentence : Courez !
Decoded sentence: À lu !

(1, 16, 70)
-
Input sentence  : Who?
Target sentence : Qui ?
Decoded sentence: Qui a parré ?

(1, 16, 70)
-
Input sentence  : Wow!
Target sentence : Ça alors !
Decoded sentence: À la fait !

(1, 16, 70)
-
Input sentence  : Fire!
Target sentence : Au feu !
Decoded sentence: À la fait !

(1, 16, 70)
-
Input sentence  : Help!
Target sentence : À l'aide !
Decoded sentence: Alle !

(1, 16, 70)
-
Input sentence  : Jump.
Target sentence : Saute.
Decoded sentence: Rentez-vous !

(1, 16, 70)
-
Input sentence  : Stop!
Target sentence : Ça suffit !
Decoded s

-
Input sentence  : Be fair.
Target sentence : Soyez équitable !
Decoded sentence: Sois prudente !

(1, 16, 70)
-
Input sentence  : Be fair.
Target sentence : Soyez équitables !
Decoded sentence: Sois prudente !

(1, 16, 70)
-
Input sentence  : Be kind.
Target sentence : Sois gentil.
Decoded sentence: Sois patent.

(1, 16, 70)
-
Input sentence  : Be nice.
Target sentence : Sois gentil !
Decoded sentence: Sois gentil !

(1, 16, 70)
-
Input sentence  : Be nice.
Target sentence : Sois gentille !
Decoded sentence: Sois gentil !

(1, 16, 70)
-
Input sentence  : Be nice.
Target sentence : Soyez gentil !
Decoded sentence: Sois gentil !

(1, 16, 70)
-
Input sentence  : Be nice.
Target sentence : Soyez gentille !
Decoded sentence: Sois gentil !

(1, 16, 70)
-
Input sentence  : Be nice.
Target sentence : Soyez gentils !
Decoded sentence: Sois gentil !

(1, 16, 70)
-
Input sentence  : Be nice.
Target sentence : Soyez gentilles !
Decoded sentence: Sois gentil !

(1, 16, 70)
-
Input sentence  : Bea